<a href="https://colab.research.google.com/github/w3slee/Android-Hooking/blob/master/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://github.com/w3slee/kadinali/blob/master/wakadinali.txt

--2023-10-14 19:00:56--  https://github.com/w3slee/kadinali/blob/master/wakadinali.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65270 (64K) [text/plain]
Saving to: ‘wakadinali.txt’

wakadinali.txt      100%[===================>]  63.74K  --.-KB/s    in 0.03s   

2023-10-14 19:00:57 (2.47 MB/s) - ‘wakadinali.txt’ saved [65270/65270]



In [4]:
# read it in to inspect it
with open('wakadinali.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  65173


In [6]:
# let's look at the first 1000 characters
print(text[:1000])

{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":"wakadinali.txt","path":"wakadinali.txt","contentType":"file"}],"totalCount":1}},"fileTreeProcessingTime":1.645678,"foldersToFetch":[],"reducedMotionEnabled":null,"repo":{"id":705004737,"defaultBranch":"master","name":"kadinali","ownerLogin":"w3slee","currentUserCanPush":false,"isFork":false,"isEmpty":false,"createdAt":"2023-10-14T18:56:41.000Z","ownerAvatar":"https://avatars.githubusercontent.com/u/52038396?v=4","public":true,"private":false,"isOrgOwned":false},"symbolsExpanded":false,"treeExpanded":true,"refInfo":{"name":"master","listCacheKey":"v0:1697310023.0","canEdit":false,"refType":"branch","currentOid":"b2802cf82cadf7b91ad0d4fec9a5f6bbd80b1a3e"},"path":"wakadinali.txt","currentUser":null,"blob":{"rawLines":["[Intro]","(Afrvka)","","[Chorus: Munga Domani]","Yeah, reputation ni same, gang ingemblein, ningekuja na ndeng'a","I can't believe this day, nilitoka jail nikakujiwa na dem","Hii ni ya day ones pekee,

In [7]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

 !"#%&'()*,-./0123456789:=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_abcdefghijklmnopqrstuvwxyz{}·úе —’… 
93


In [9]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("tuko area"))
print(decode(encode("tuko area")))

[76, 77, 67, 71, 0, 57, 74, 61, 57]
tuko area


In [10]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([65173]) torch.int64
tensor([83,  2, 72, 57, 81, 68, 71, 57, 60,  2, 24, 83,  2, 57, 68, 68, 45, 64,
        71, 74, 76, 59, 77, 76, 75, 31, 70, 57, 58, 68, 61, 60,  2, 24, 62, 57,
        68, 75, 61, 10,  2, 62, 65, 68, 61, 46, 74, 61, 61,  2, 24, 83,  2,  2,
        24, 83,  2, 65, 76, 61, 69, 75,  2, 24, 53, 83,  2, 70, 57, 69, 61,  2,
        24,  2, 79, 57, 67, 57, 60, 65, 70, 57, 68, 65, 12, 76, 80, 76,  2, 10,
         2, 72, 57, 76, 64,  2, 24,  2, 79, 57, 67, 57, 60, 65, 70, 57, 68, 65,
        12, 76, 80, 76,  2, 10,  2, 59, 71, 70, 76, 61, 70, 76, 46, 81, 72, 61,
         2, 24,  2, 62, 65, 68, 61,  2, 84, 55, 10,  2, 76, 71, 76, 57, 68, 29,
        71, 77, 70, 76,  2, 24, 15, 84, 84, 10,  2, 62, 65, 68, 61, 46, 74, 61,
        61, 42, 74, 71, 59, 61, 75, 75, 65, 70, 63, 46, 65, 69, 61,  2, 24, 15,
        12, 20, 18, 19, 20, 21, 22, 10,  2, 62, 71, 68, 60, 61, 74, 75, 46, 71,
        32, 61, 76, 59, 64,  2, 24, 53, 55, 10,  2, 74, 61, 60, 77, 59, 61, 60,
        

In [11]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 8
train_data[:block_size+1]

tensor([83,  2, 72, 57, 81, 68, 71, 57, 60])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([83]) the target: 2
when input is tensor([83,  2]) the target: 72
when input is tensor([83,  2, 72]) the target: 57
when input is tensor([83,  2, 72, 57]) the target: 81
when input is tensor([83,  2, 72, 57, 81]) the target: 68
when input is tensor([83,  2, 72, 57, 81, 68]) the target: 71
when input is tensor([83,  2, 72, 57, 81, 68, 71]) the target: 57
when input is tensor([83,  2, 72, 57, 81, 68, 71, 57]) the target: 60


In [15]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[69, 57, 74, 63, 57, 74, 65, 70],
        [ 2, 27, 70, 57, 72, 61, 70, 60],
        [82, 65, 10,  0, 68, 65, 62, 61],
        [ 2, 10,  2, 27, 70, 57, 60, 57]])
targets:
torch.Size([4, 8])
tensor([[57, 74, 63, 57, 74, 65, 70, 61],
        [27, 70, 57, 72, 61, 70, 60, 57],
        [65, 10,  0, 68, 65, 62, 61,  0],
        [10,  2, 27, 70, 57, 60, 57, 65]])
----
when input is [69] the target: 57
when input is [69, 57] the target: 74
when input is [69, 57, 74] the target: 63
when input is [69, 57, 74, 63] the target: 57
when input is [69, 57, 74, 63, 57] the target: 74
when input is [69, 57, 74, 63, 57, 74] the target: 65
when input is [69, 57, 74, 63, 57, 74, 65] the target: 70
when input is [69, 57, 74, 63, 57, 74, 65, 70] the target: 61
when input is [2] the target: 27
when input is [2, 27] the target: 70
when input is [2, 27, 70] the target: 57
when input is [2, 27, 70, 57] the target: 72
when input is [2, 27, 70, 57, 72] the target: 61
when input is

In [16]:
print(xb) # our input to the transformer

tensor([[69, 57, 74, 63, 57, 74, 65, 70],
        [ 2, 27, 70, 57, 72, 61, 70, 60],
        [82, 65, 10,  0, 68, 65, 62, 61],
        [ 2, 10,  2, 27, 70, 57, 60, 57]])


In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 93])
tensor(5.0017, grad_fn=<NllLossBackward0>)
 xGr)…]INyyO',Mz/m(Aе[)s?Hr{p]rQúK…PA7T*m]2W.A? 4LVmOww·rqF xUol4huXH#C’jqkOM\}rt!0z8RQfhp{*l8ncUmu{:


In [18]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [21]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.642594337463379


In [22]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

 9%W9ne9n5VIH-Eai=j.-G )z#H69…2—GuafkQSlojQV?Su·n5…caеQjBmaBEAiAZ]3WRDiIU_'ú==,358V f{G-е’\_aIUrw—MVH= 5unR{VCoxfGI]Bst·ú5qHú:w#x9[ym’uphtaH6fLea5Bú}.Z *zHXznúX45qlJn 9-cu·",Ty,!=F6Pf_CеCMt]M{zZjnB b.ZFh&—/R—MVha\?Yqtv3_AC)S5llsa \=5Bc\o4(0no7nNy,/lDz:-M:?—Gay ZO8\Qs5Di/!SPO—yf)qúE:{&·!lb_=Fu!bKY/rPO}kJD7*WH=HеRF3X-d*cj4Z=b{údX&4?—h&·E2f9wVcgu·K#qR 2E0jSFoo8mKJ{9/Ezuh&Yplstkúl27{pyNLLp/aiwxK Lph*W0·%Pe·cR7(uG(diVmnqX,C ,Cm!tA?T·0Ewj)BE:Pz…GWqBNqkI,:m}{=F EHzcJGWbn) ea,…-gu?!Mq]o6,l—oHP9ddI]f&AkSL


## The mathematical trick in self-attention

In [23]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [24]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [25]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [26]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [27]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [28]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [29]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [30]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [31]:
k.var()

tensor(1.0449)

In [32]:
q.var()

tensor(1.0700)

In [33]:
wei.var()

tensor(1.0918)

In [34]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [35]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [36]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [37]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [38]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [41]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# rada ni safi <START> rada safi bro<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [44]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://github.com/w3slee/kadinali/blob/master/wakadinali.txt
with open('wakadinali.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.213341 M parameters
step 0: train loss 4.6834, val loss 4.6424
step 100: train loss 2.6365, val loss 2.0893
step 200: train loss 2.4998, val loss 2.0192
step 300: train loss 2.4470, val loss 2.0243
step 400: train loss 2.3921, val loss 1.9875
step 500: train loss 2.3407, val loss 2.0035
step 600: train loss 2.2903, val loss 1.9783
step 700: train loss 2.2630, val loss 1.9349
step 800: train loss 2.2170, val loss 1.8911
step 900: train loss 2.1615, val loss 1.9255
step 1000: train loss 2.1336, val loss 1.8950
step 1100: train loss 2.1076, val loss 1.8395
step 1200: train loss 2.0537, val loss 1.8359
step 1300: train loss 2.0117, val loss 1.8145
step 1400: train loss 1.9762, val loss 1.8574
step 1500: train loss 1.9634, val loss 1.8304
step 1600: train loss 1.9091, val loss 1.7951
step 1700: train loss 1.8756, val loss 1.9154
step 1800: train loss 1.8170, val loss 1.8088
step 1900: train loss 1.7848, val loss 1.8196
step 2000: train loss 1.7707, val loss 1.8409
step 2100: train loss 1.